# 3. Part-of-Speech (PoS) Tagging

Tractament de Dades Textuals i Codificades — Eixample Clínic, 2026

Pol Pastells, ppastells@eixampleclinic.es

---

Avui veurem com realitzar l'etiquetatge Part-of-Speech (PoS) utilitzant NLTK. 

Farem servir diferents models segons l'n-grama triat i veurem l'importància de seleccionar un bon corpus d'entrenament.

Presentarem també les mètriques i tècniques d'avaluació bàsiques.

# Teoria

## 1. Què és el Part-of-Speech?
Una **Part-of-Speech (PoS)** o categoria gramatical és una etiqueta que s'assigna a cada paraula d'acord amb la seva funció dins d'una oració.

*   **Conjunts d'etiquetes (Tagsets):**
    *   **Simplificats:** Únicament marquen la categoria principal (Ex: "nom", "verb", "adjectiu").
    *   **Complexos:** Contenen característiques gramaticals detallades (Ex: número, persona, forma verbal, gènere).
*   **Evolució:**
    *   Originalment, els conjunts d'etiquetes eren específics per a cada llengua.
    *   Amb el desenvolupament de la lingüística computacional i el NLP, es van introduir els **conjunts d'etiquetes universals**. Podeu consultar-los a: [Universal Dependencies](https://universaldependencies.org/).

## 2. El procés d'etiquetatge (PoS Tagging)
El PoS Tagging és el procés automàtic d'assignar una etiqueta a cada paraula d'un text:
*   *Jugant*: 'verb'
*   *Cotxe*: 'nom'

### El problema de l'ambigüitat

Moltes paraules poden tenir diferents categories segons l'ús. Per exemple:

- cura (NOM): remei o tractament.
- cura (VERB): acció de guarir (del verb curar).

Per desambiguar, necessitem observar el context:

1. "El metge ha trobat una cura per a la malaltia."  →  NOM
2. "Aquest medicament cura la infecció ràpidament."  →  VERB

Un altre exemple molt comú:

- baixa (NOM): document d'incapacitat laboral.
- baixa (VERB): acció de descendir.
- baixa (ADJECTIU): de poca altura o intensitat.

Contextualització:

1. "L'infermer li ha tramitat la baixa."  →  NOM
2. "El pacient baixa amb l'ascensor."  →  VERB
3. "Té la pressió arterial molt baixa."  →  ADJECTIU

**Com analitzem el context?**
*   **Context Lineal:** Observem un número limitat de paraules abans i després (n-grames).
*   **Per què no fem anàlisi sintàctic directament?** L'anàlisi sintàctic és molt més complex; generalment s'obtenen les etiquetes PoS com a pas previ necessari.

---

## 3. Etiquetatge basat en N-grames
Aquests models prediuen l'etiqueta d'una paraula basant-se en la paraula actual i un context predefinit de paraules adjacents.

### Elecció de l'N-grama:
*   **Unigrames:** Prediu l'etiqueta utilitzant únicament la paraula actual. Si al corpus "cura" apareix el 90% de cops com a nom, sempre l'etiquetarà com a nom.
*   **Bigrames:** Prediu l'etiqueta basant-se en la paraula actual i l'anterior (*"la cura"* $\rightarrow$ NOM; *"medicament cura"* $\rightarrow$ VERB).
*   **Trigrames:** Utilitza 3 paraules consecutives per predir l'etiqueta.
*   **N-grames de major ordre:** Ofereixen més context (més potència) però pateixen el problema de l'**escassetat de dades (data sparsity)**: és difícil trobar seqüències llargues exactes en el corpus d'entrenament.

---

## 4. Entrenament i Avaluació
El PoS Tagging és un exemple d'**Aprenentatge Supervisat (Supervised Learning)**:

1.  **Corpus Anotat:** Necessitem un conjunt de dades ja etiquetat per humans.
2.  **Entrenament:** El model calcula les freqüències i probabilitats de cada paraula i cada seqüència (n-grama).
3.  **Avaluació:** Provem el model amb dades que no ha vist mai per comprovar-ne la precisió (**Accuracy**). És vital separar el corpus en **Train** (entrenament) i **Test** (avaluació).

### Possibles problemes:
*   **Qualitat del corpus:** Si "baixa" només apareix com a verb al corpus d'entrenament, el model mai el marcarà com a nom.
*   **Paraules desconegudes (Out-of-Vocabulary):** Si una paraula no apareix al training, el model no sabrà què fer. Solució: analitzar prefixos/sufixos (**AffixTagger**).

---

## 5. La tècnica del Backoff
El **Backoff** és una estratègia per compensar l'escassetat de dades combinant múltiples etiquetadors en cascada:

1.  Intentem etiquetar amb el model més precís (ex: **Trigrames**).
2.  Si el trigrama no coneix la combinació, "baixem" a un model de context menor (**Bigrames**).
3.  Si aquest tampoc, baixem a **Unigrames**.
4.  Com a última instància, usem un **DefaultTagger** (que etiqueta tot com a Substantiu, per exemple).

---

## Pràctica d'avui
En aquesta sessió:
*   Explorarem diferents etiquetadors de la llibreria **NLTK**.
*   Comprovarem l'efectivitat del **Backoff**.
*   Veurem com responen els models davant de vocabulari general vs. vocabulari mèdic/específic.

# Codi

In [ ]:
# Importar nltk
import nltk

In [ ]:
# Descarreguem els paquets importants per la sessió d'avui
nltk.download("brown")
nltk.download("universal_tagset")

In [ ]:
# Importem els recursos NLTK necessaris i la resta de llibreries:
# Importar etiquetadores
from nltk import (
    AffixTagger,
    BigramTagger,
    ClassifierBasedPOSTagger,
    ConfusionMatrix,
    DefaultTagger,
    TrigramTagger,
    UnigramTagger,
    bigrams,
    trigrams,
)

# Importar el corpus brown
from nltk.corpus import brown

## Etiquetatge PoS a Python

A continuació repassarem com podem accedir al contingut del corpus amb què treballarem. Tenim les frases etiquetades, paraula a paraula, amb les seves categories. Prèviament, s'han hagut de tokenitzar (recordem la sessió passada).

Ens centrarem en els models 'taggers' basats en n-grames

In [ ]:
# Obtenim una llista amb les categories del corpus brown
brown.categories()

In [ ]:
# Obtenim la versió tokenitzada i etiquetada de la categoria 'news'
brown_twords = brown.tagged_words(categories="news")

# Obtenim la mateixa versió que abans, però a més, segmentada per frases
brown_tsents = brown.tagged_sents(categories="news")

In [ ]:
# Obtenim les 5 primeres paraules etiquetades
print("\nLes primeres 5 paraules de la versió tokenitzada i etiquetada són:")
print(brown_twords[:5])

In [ ]:
# Observem les dues primeres frases
print("\nLes primeres 2 frases de la versió amb frases segmentades són:")
print(brown_tsents[:2])

In [ ]:
# Obtenim el conjunt de totes les etiquetes que apareixen en el corpus brown
brown_tags = set([tag for (token, tag) in brown_twords])
print("\nEl conjunt d'etiquetes en el corpus brown és:")
print(len(brown_tags), brown_tags)

Veiem que n'hi ha un munt. El corpus __Brown__ està anotat amb molta precisió. Si volem comparar les etiquetes amb les d'Universal Dependencies, podem fer servir `tagset="universal"`.

### Incís

Cada corpus pot tenir un seguit d'etiquetes diferents.
- Les d'Universal Dependencies les podeu trobar a: https://universaldependencies.org/u/pos/index.html
- Les etiquetes que fa servir NLTK (almenys per l'anglès) són les del Penn TreeBank https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html
- La relació entre Penn i Universal Dependencies la podeu trobar a https://universaldependencies.org/tagset-conversion/en-penn-uposf.html
- Les etiquetes del corpus Brown són https://varieng.helsinki.fi/CoRD/corpora/BROWN/tags.html, a més s'utilitzen sufixos per indicar el context, e.g., títol (-TL), i _+_ simbolitza formes contretes (I'm).

In [ ]:
# Ara obtenim i visualitzem totes les etiquetes existents en el set 'universal'
brown_utwords = brown.tagged_words(categories="news", tagset="universal")
universal_tags = set([tag for (token, tag) in brown_utwords])
print("\nEl conjunt d'etiquetes universals és:")
print(len(universal_tags), universal_tags)

### Tipus d'etiquetadors

Coneixent ja com accedir al contingut etiquetat del corpus, entrenarem i avaluarem els diferents models d'etiquetatge:

- **default** (etiquetatge per defecte)
- **affix** (etiquetatge per prefixos/sufixos)
- **unigram** (etiquetatge basat en un sol token)
- **bigram** (etiquetatge basat en conjunts de dos tokens)
- **trigram** (etiquetatge basat en conjunts de tres tokens)

### Mètrica d'avaluació

Per avaluar els models, farem servir la mètrica **Accuracy** (precisió), que es formula de la següent manera:


$$
    Accuracy = \frac{\# Prediccions Correctes}{\# Prediccions} 
$$

El valor d'accuracy oscil·larà del 0 al 1, sent 0 una detecció nul·la, i 1 una molt bona precisió del model. Tenir un valor prop del 0.5 voldrà dir que el model prediu quasi com llençant una moneda a cara o creu...

## Entrenament i avaluació

In [ ]:
# TAGSET = "universal"
TAGSET = None  # per defecte, etiquetes completes

### Etiquetador per defecte

In [ ]:
# Obtenim la versió segmentada i tokenitzada de 'news'
brown_sents = brown.sents(categories="news")
# així com les frases etiquetades,
brown_tsents = brown.tagged_sents(categories="news", tagset=TAGSET)

# Obtenim una llista de totes les etiquetes del corpus
tags = [tag for (word, tag) in brown.tagged_words(categories="news", tagset=TAGSET)]
print("Hi ha un total de {} tags (mostres en el conjunt dataset)\n".format(len(tags)))

# Desem l'etiqueta més freqüent del corpus
most_frequent_tag = nltk.FreqDist(tags).max()
print("l'etiqueta més freqüent és:", most_frequent_tag)

Configurem un etiquetador per defecte.
Assignarà la mateixa etiqueta per defecte a tots els tokens del corpus.
El configurem perquè posi l'etiqueta més freqüent a tots els tokens.

In [ ]:
default_tagger = nltk.DefaultTagger(most_frequent_tag)

my_sent = "the quick brown fox jumped over the lazy dog".split()
print("Etiquetatge per defecte:")
print(default_tagger.tag(my_sent))

In [ ]:
# Evaluem l'etiquetador per defecte en el corpus:
print(
    "La precisió de l'etiquetador per defecte en el corpus Brown es:",
    round(default_tagger.accuracy(brown_tsents), 2),
)

Per la resta d'etiquetadors, dividirem el corpus en 'train' i 'test':

In [ ]:
brown_tsents = brown.tagged_sents(categories="news", tagset=TAGSET)

test_corpus = brown_tsents[:800]
train_corpus = brown_tsents[800:]

print(test_corpus[0])

### AffixTagger

És un etiquetador que es basa en prefixos i sufixos. S'entrena tenint en compte una longitud fixa de principi o final de paraula que definim prèviament.
Variant el paràmetre `affix_length` canviem què es consiedera una affix. Un valor positiu serà un prefix, negatiu un sufix. Valor negatiu busca sufixos.

In [ ]:
# Entrena el etiquetador affix
affix_tagger = AffixTagger(train_corpus, affix_length=2)
# prova a canviar l'affix_lenght a veure si canvia el resultat

# Etiqueta el corpus
affix_sents = affix_tagger.tag_sents(test_corpus)

# Imprimim la primera frase i l'accuracy del model:
print("\nLa primera frase etiquetada amb l'affix tagger és:")
print(affix_sents[0])

print("\nL'accuracy de l'affix tagger és:", round(affix_tagger.accuracy(test_corpus), 2))

#### Exemple amb paraules del món clínic

In [ ]:
# Creem un corpus d'entrenament, dividit per "famílies" de sufixos
# El format ha de ser una llista de llistes de tuples (simulant frases)
train_medical = [
    [
        # Sufixos de malaltia: -itis (4 lletres) -> Substantiu
        ("gastritis", "NN"),
        ("arthritis", "NN"),
        ("hepatitis", "NN"),
        ("nephritis", "NN"),
        ("dermatitis", "NN"),
        # Sufixos d'especialitat: -logy / -logia (4 lletres) -> Substantiu
        ("cardiology", "NN"),
        ("neurology", "NN"),
        ("oncology", "NN"),
        ("pathology", "NN"),
        ("hematology", "NN"),
        # Sufixos d'adjectius: -tric (4 lletres) -> Adjectiu
        ("geriatric", "JJ"),
        ("psychiatric", "JJ"),
        ("pediatric", "JJ"),
        ("obstetric", "JJ"),
    ]
]

# Entrenem l'AffixTagger
# Fem servir affix_length=-4 perquè molts sufixos mèdics tenen 4 lletres
suffix_tagger = AffixTagger(train_medical, affix_length=-4)

# Testejem amb paraules que NO estan a train_medical
test_words = ["bronchitis", "radiology", "pediatric"]

print("Resultats per sufixos (-4 lletres):")
for word in test_words:
    tag = suffix_tagger.tag([word])
    print(f"{word} -> {tag}")

### N-gram taggers

In [ ]:
# Entrenem amb el mateix corpus de 'train' un unigram tagger:
unigram_tagger = UnigramTagger(train_corpus)

# Etiquetem corpus per visualitzar resultat
uni_sents = unigram_tagger.tag_sents(test_corpus)

# Visualitzem
print("\nLa primera frase etiquetada amb l'unigram tagger és:")
print(uni_sents[0])

# Mesurem precisió amb el corpus 'test'
print("\nL'accuracy de l'unigram tagger és:", round(unigram_tagger.accuracy(test_corpus), 2))

In [ ]:
# Entrenamient d'un bigram tagger
bigram_tagger = BigramTagger(train_corpus)

# Etiquetem
bi_sents = bigram_tagger.tag_sents(test_corpus)

# Visualitzem
print("\nPrimera frase etiquetada amb el model PoS tagger basat en bigrames és:")
print(bi_sents[0])

# Avaluem
print(
    "\nLa precisió de l'etiquetador basat en bigrames en el corpus és:",
    round(bigram_tagger.accuracy(test_corpus), 2),
)

In [ ]:
# Entrenament d'un trigram tagger:
trigram_tagger = TrigramTagger(train_corpus)

# Etiquetat
tri_sents = trigram_tagger.tag_sents(test_corpus)

# Visualització
print("\nPrimera frase etiquetada amb el model PoS tagger basat en trigrames és:")
print(tri_sents[0])

# Avaluació
print(
    "\nLa precisió de l'etiquetador basat en trigrames en el corpus és:",
    round(trigram_tagger.accuracy(test_corpus), 2),
)

#### Apliquem el model 'backoff'

In [ ]:
# Entrenament d'un bigram tagger amb backoff definit com un unigram tagger (l'anterior entrenat)
bigram_tagger_backoff = BigramTagger(train_corpus, backoff=unigram_tagger)

# Etiquetat per veure la diferència
bi_sents_bo = bigram_tagger_backoff.tag_sents(test_corpus)

# Visualitzem
print("\nLa primera frase etiquetada amb l'etiquetador basat en bigrames amb backoff és:")
print(bi_sents_bo[0])

# Avaluació
print(
    "\nLa precisió de l'etiquetador basat en bigrames amb backoff en el corpus és:",
    round(bigram_tagger_backoff.accuracy(test_corpus), 2),
)

### Tasca 1

Entrena i avalua en el mateix corpus, en comptes de fer servir una partició de _test_ separada. Quin es el resultat?

### Tasca 2

Prova d'avaluar un etiquetador en un domini diferent a l'entrenat (cross-domain).

Per exemple, entrena al corpus "news" i avalua al corpus "science_fiction", compara-ho amb entrenar i avaluar al mateix corpus "science_fiction".

#### EXTRA

Fes una taula (e.g., llista de llistes o CSV) on es mostrin totes les combinacions de diferents categories. 

Si ho fas amb totes les categories és possible que tardi bastant. Prova-ho amb 3 o 4 categories, però intenta fer el codi genèric.

### Tasca 3

Entreneu i avalueu els mateixos models tagger que hem vist però amb tot el corpus Brown (sense `categories=`) separat per 'train' i 'test'.

Dividiu (aproximadament) 'train' 80% i 'test' 20% del total de dades.

### Tasca 4
Crea una seqüència d'etiquetadors a mode de backoff:
trigram -> bigram -> unigram -> affix -> default

Utilitzeu tot el corpus brown i separeu en 'train' i 'test'.

Avalua l'etiquetador resultant.

### Tasca 5 - Importància del conjunt d'etiquetes
Prova a refer algun dels exercicis anteriors amb les etiquetes "universal"